<a href="https://colab.research.google.com/github/Flammingo2/TensorFlow/blob/main/homework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

In [30]:
# Load the dataset
train_dataset, test_dataset = tfds.load('genomics_ood', split = ['train', 'test'], as_supervised = True, shuffle_files = True)

#Slicing datasets into images and labels
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_targets))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_targets))

#Look at shapes
print(train_images.shape)
print(train_targets.shape)
print(test_images.shape)
print(test_targets.shape)

print(np.max(train_images[0]))
print(np.min(train_images[0]))

#Reshape to 1D tensors
# 1. train dataset
train_images = train_images.map(lambda img : tf.reshape(img, (-1,)))
train_targets = train_targets.map(lambda t : tf.one_hot(t, 10))
# 2. test dataset
test_images = test_images.map(lambda img : tf.reshape(img, (-1,)))
test_targets = test_targets.map(lambda t : tf.one_hot(t,10))

#Zip together input images and targets
train_dataset = tf.data.Dataset.zip((train_images, train_targets)).batch(128).shuffle(buffer_size=128)

NameError: ignored